In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

# Initialize lemmatizer and load intents and model
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

# Load preprocessed data and model
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

# Function to clean up the input sentence
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function to create a bag of words
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

# Function to predict the class of the input
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)

    # Ensure there's at least one result; if not, set a default response
    if not results:
        return [{'intent': 'no_match', 'probability': '1.0'}]  # Add a 'no_match' intent with default response in intents.json
    
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    
    return return_list

# Function to get the response based on the predicted intent
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            # Randomly select a response to improve variability
            result = random.choice(i['responses'])
            break
    return result

print("GO! Bot is running!")

# Main loop to keep the bot running
while True:
    message = input("You: ")
    ints = predict_class(message)
    # Check if prediction is empty and handle appropriately
    if ints:
        res = get_response(ints, intents)
        print(f"Bot: {res}")
    else:
        print("Bot: I'm sorry, I didn't understand that. Could you please rephrase?")
